# Processamento de Linguagem Natural aplicada à Gestão Pública

Aula 5 (08/06): Modelagem de Tópicos, Classificação de Textos e Reconhecimento de entidades nomeadas

Reconhecimento de menção a pessoas, lugares e organizações e outras entidades. Classificação de Textos. Identificação de tópicos. Alocação Latente de Dirichlet.



## Preparação do ambiente

In [ ]:
# Download dos dados
%%capture
!pip install spacy==3.3.0
!git clone https://github.com/samuelbarbosaa/oficina_nlp.git

import gdown
import tarfile
import spacy

url = 'https://drive.google.com/uc?id=1-zEjxcpyd3OwBI1Uve90MMcIkjRPH7FR'
output = 'pt_core_news_lg-3.3.0.tar.gz'
gdown.download(url, output, quiet=False)

file = tarfile.open('pt_core_news_lg-3.3.0.tar.gz')
file.extractall('./')
file.close()

nlp = spacy.load("/content/pt_core_news_lg-3.3.0/pt_core_news_lg/pt_core_news_lg-3.3.0")

In [ ]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.options.display.max_rows = 6
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
import zipfile

with zipfile.ZipFile("/content/oficina_nlp/data/acordaos_cpf.zip", "r") as zip_ref:
    zip_ref.extractall("/content/oficina_nlp/data/")

acordaos = pd.read_csv("/content/oficina_nlp/data/acordaos_cpf.csv")
acordaos

In [ ]:
import nltk
import string
import re
from spacy.lang.pt import STOP_WORDS as spacy_stopwords
from functools import reduce

nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words("portuguese")
all_sw = list(spacy_stopwords) + nltk_stopwords

def remove_pontuacao(x):
    punct = [s for s in string.punctuation] + ["º"]
    return reduce(lambda w, s: w.replace(s, ""), punct, x)

def remove_stopwords(x):
    palavras = x.split(" ")
    return " ".join([w for w in palavras if w.lower() not in all_sw])

def remove_espacos_extras(x):
    return re.sub(r"\s+", " ", x)

def remove_ngram(x, n):
    palavras = x.split(" ")
    return " ".join([w for w in palavras if not len(w) <= n])

def padronizar_texto(x):
    y = re.sub(pattern=r"\W", repl=" ", string=x)
    y = re.sub(pattern=r"\d", repl="", string=y)
    y = re.sub(pattern=r"\n\r+", repl="", string=y)
    y = remove_pontuacao(y)
    y = remove_stopwords(y)
    y = remove_espacos_extras(y)
    y = remove_ngram(y, 2)
    return y.lower()

# REMOVER NOMES PROPRIOS

In [ ]:
#acordaos["acordao_padr"] = acordaos["acordao"].apply(padronizar_texto)

In [ ]:
#docs = list(nlp.pipe(acordaos["acordao_padr"], disable=['parser', 'ner']))

#def lemmatizar(doc):
#  lemmas = [w.lemma_ for w in doc if w.pos_ != "PROPN"]
#  return " ".join(lemmas)

#acordaos["acordao_padr"] = [lemmatizar(doc) for doc in docs]
#acordaos

In [ ]:
import pickle

#with open("/content/oficina_nlp/acordaos.pkl", "wb") as fp:
#  pickle.dump(acordaos, fp)

with open("/content/oficina_nlp/acordaos.pkl", "rb") as fp:
  acordaos = pickle.load(fp)

acordaos

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

metodo = TfidfVectorizer()
modelo = metodo.fit(acordaos["acordao_padr"])
vocab = modelo.get_feature_names_out()
TF_IDF = modelo.transform(acordaos["acordao_padr"])
TF_IDF

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=30, max_iter=200)
topicos = lda.fit_transform(TF_IDF)
palavras_topicos = lda.components_

with open("/content/oficina_nlp/lda.pkl", "wb") as fp:
  pickle.dump(lda, fp)

with open("/content/oficina_nlp/lda.pkl", "rb") as fp:
  lda = pickle.load(fp)


In [11]:
%%capture
!pip install pyldavis

In [12]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, TF_IDF, metodo, mds="tsne", R=15)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/p

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
15     -10.878068   10.538728       1        1  25.938135
11     -60.884026  -75.591820       2        1  17.784024
2       -2.336119  -53.289074       3        1  10.464057
...           ...         ...     ...      ...        ...
14      47.588097  121.934814      18        1   0.896930
13    -106.467834   30.176889      19        1   0.875021
9      -50.613804   59.328842      20        1   0.859785

[20 rows x 5 columns], topic_info=           Term         Freq        Total Category  logprob  loglift
646    admissão  2354.000000  2354.000000  Default  30.0000  30.0000
18872    inciso  3421.000000  3421.000000  Default  29.0000  29.0000
3218        ato  2305.000000  2305.000000  Default  28.0000  28.0000
...         ...          ...          ...      ...      ...      ...
11566     dório     0.999078     2.528416  Topic20  -7.4080   3.8277
34615    sotero     0.996549     2.816307  Topic20  -7.4105   3.7174
26121    noelia     0.827541     1.887062  Topic20  -7.5963   3.9319

[1097 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
18         1  0.922751  abaixo
18         2  0.031445  abaixo
18         3  0.001084  abaixo
...      ...       ...     ...
39468      8  0.021650   órgão
39468      9  0.001665   órgão
39468     10  0.006661   órgão

[2264 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 12, 3, 6, 1, 19, 5, 7, 4, 9, 17, 11, 18, 2, 20, 13, 8, 15, 14, 10])